In [1]:
import pandas as pd
import numpy as np
from keras.layers import Input, Dense, LSTM, RepeatVector, TimeDistributed, RNN, Lambda, Concatenate, Reshape
from keras.layers.merge import Multiply
from keras.models import Model
from keras.losses import mean_squared_error, binary_crossentropy
from keras.models import Model
from keras.models import load_model
from keras.models import model_from_json
import keras.backend as K
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error as mse
def open_and_load(string):
    with open('trained/'+string+'.json', 'r') as f:
        model = model_from_json(f.read())
    model.load_weights('trained/'+string+'.h5')   
    return model

Using TensorFlow backend.


In [2]:
# i = Input(shape=(1,))
# h = Dense(1, activation='relu')(i)
# h = Dense(1, activation='tanh')(h)
# change = Model(inputs=i, outputs=h)
# change.compile(loss=mean_squared_error,
#               optimizer='adam',
#               metrics=['accuracy'])
# # change.summary()

# 1

In [2]:
def change_gen(batch_size=128):
    while True:
        a = np.random.randn(batch_size)
        b = (a > 0)
        yield a, b        

In [3]:
change.fit_generator(change_gen(), steps_per_epoch=128000, epochs=3)

NameError: name 'change' is not defined

In [4]:
i = Input(shape=(1,))
h = Dense(1, activation='relu')(i)
h = Dense(1, activation='hard_sigmoid')(h)
change2 = Model(inputs=i, outputs=h)
change2.compile(loss=mean_squared_error,
              optimizer='adam')
# change.summary()
change2.fit_generator(change_gen(), steps_per_epoch=128000, epochs=3)

Epoch 1/3


InternalError: Blas GEMV launch failed:  m=1, n=128
	 [[{{node dense_1/MatMul}} = MatMul[T=DT_FLOAT, _class=["loc:@training/Adam/gradients/dense_1/MatMul_grad/MatMul_1"], transpose_a=false, transpose_b=false, _device="/job:localhost/replica:0/task:0/device:GPU:0"](_arg_input_1_0_0/_47, dense_1/kernel/read)]]
	 [[{{node loss/mul/_67}} = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_404_loss/mul", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

In [ ]:
change2.predict([1,-1,-0.5,0.1, 0.0005, -0.0005])

In [8]:
save_model(change2, 'change')

Saved model to disk


In [9]:
t = np.random.randn(120)
P = change2.predict(t)[:,0]
mse(P, (t>0))

0.0

In [10]:
P.shape, t.shape

((120,), (120,))

In [13]:
# P==(t>0)

# 2

In [6]:
i = Input(shape=(3,))
h = Dense(2, use_bias=False)(i) #mby act=none
h = Dense(1, activation='hard_sigmoid', use_bias=False)(h)
h = Lambda(lambda s: K.concatenate([1-s,s]), output_shape=(2,))(h)
border_att = Model(inputs=i, outputs=h)
border_att.compile(loss=mean_squared_error,
              optimizer='adam',
              metrics=['accuracy'])
border_att.summary()
# border.fit_generator(border_gen(), steps_per_epoch=128000, epochs=3, )

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 3)                 0         
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 6         
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 2         
_________________________________________________________________
lambda_1 (Lambda)            (None, 2)                 0         
Total params: 8
Trainable params: 8
Non-trainable params: 0
_________________________________________________________________


In [81]:
def border_att_gen(batch_size=128):
    while True:
        a = np.zeros((batch_size,3) )
        b1 = np.random.uniform(low=-10,high=0,size=batch_size)
        b2 = np.random.uniform(low=0,high=10,size=batch_size)
        x = np.random.uniform(low=b1, high=b2)
        a[:,0] = b1
        a[:,1] = b2
        a[:,2] = x
        
        isb1 = ((x-b1)/(b2-b1) < 0.5)
        
        yield a, np.array([isb1,(1-isb1)]).T

In [82]:
border_att.trainable=True

In [83]:
border_att.fit_generator(border_att_gen(), steps_per_epoch=128000, epochs=2)

Epoch 1/2
128000/128000 [==============================] - 164s 1ms/step - loss: 0.0039 - acc: 0.9985
Epoch 2/2
128000/128000 [==============================] - 165s 1ms/step - loss: 0.0011 - acc: 0.9995


In [84]:
a,b = next(border_att_gen(batch_size=1000))
# b = b[:,1]
P = border_att.predict(a)
mse(P,b)**0.5

0.0359551052598125

In [85]:
def border_gen(batch_size=128):
    while True:
        a = np.zeros((batch_size,3) )
        b1 = np.random.uniform(low=-10,high=0,size=batch_size)
        b2 = np.random.uniform(low=0,high=10,size=batch_size)
        x = np.random.uniform(low=b1, high=b2)
        a[:,0] = b1
        a[:,1] = b2
        a[:,2] = x
        
        isb1 = ((x-b1)/(b2-b1) < 0.5)
        
        yield a, isb1 * b1 + (1-isb1) * b2 

In [88]:
border_att.trainable = False#True#False

In [62]:
from keras.regularizers import l2

In [7]:
i = Input(shape=(3,))
border_att.name = 'attention'
h = border_att(i)

b1b2 = Lambda(lambda s: s[:,:2], output_shape=(2,), name='b1b2')(i)
mul = Multiply()([h, b1b2]) 

out = Lambda(lambda s: s[:,0]+s[:,1], output_shape=(1,))(mul)
# out = Dense(1, use_bias=False)(mul)
border = Model(inputs=i, outputs=out)
border.compile(loss=mean_squared_error,
              optimizer='adam',
              metrics=['accuracy'])
border.summary()
# border.fit_generator(border_gen(), steps_per_epoch=128000, epochs=3, )

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 3)            0                                            
__________________________________________________________________________________________________
attention (Model)               (None, 2)            8           input_4[0][0]                    
__________________________________________________________________________________________________
b1b2 (Lambda)                   (None, 2)            0           input_4[0][0]                    
__________________________________________________________________________________________________
multiply_1 (Multiply)           (None, 2)            0           attention[1][0]                  
                                                                 b1b2[0][0]                       
__________

In [8]:
from keras.utils import plot_model
plot_model(border, to_file='border.png')

In [78]:
border.fit_generator(border_gen(), steps_per_epoch=128000, epochs=5)

Epoch 1/5
 40538/128000 [========>.....................] - ETA: 1:59 - loss: 49.8765 - acc: 2.1079e-08

KeyboardInterrupt: 

In [89]:
a,b = next(border_gen(batch_size=1000))
P = border.predict(a)
mse(P,b)**0.5

0.22865992606747543

In [100]:
np.mean((P-b)**2)**0.5

0.22865992606747543

In [120]:
a.shape

()

In [116]:
b[194:195]

array([-2.61357508])

In [117]:
P[194]

0.26139867

In [112]:
for i in range(10):
    a = np.mean((P[100+i*10:100+(i+1)*10]-b[100+i*10:100+(i+1)*10])**2)**0.5
#     a = np.mean((P[i*100:(i+1)*100]-b[i*100:(i+1)*100])**2)**0.5
    print(i, ': ', a)

0 :  1.6645554173849507e-07
1 :  1.376510604716685e-07
2 :  1.9612146105764953e-07
3 :  1.1037717750305573e-07
4 :  2.1068265455961892e-07
5 :  8.807812871209753e-08
6 :  1.2609120162605079
7 :  2.299024315508637e-07
8 :  0.5728133954992445
9 :  0.9091465272835322


In [16]:
def save_model(model, model_str):
    model_json = model.to_json()
    with open('trained/'+model_str+".json", "w") as json_file:
        json_file.write(model_json)
    # serialize weights to HDF5
    model.save_weights('trained/'+model_str+".h5")
    print("Saved model to disk")

In [122]:
# save_model(border_att, 'border_att')
# save_model(border, 'border')

Saved model to disk
Saved model to disk


# 3

In [2]:
def bounce_gen(batch_size=128):
    while True:
        M = np.zeros((batch_size, 3))
        b = np.random.uniform(-10,10,size=batch_size)
        v = np.sign(b) * np.random.uniform(0, 0.1, size=batch_size)
        x = np.random.uniform(b-v, b)
        M[:,0],M[:,1],M[:,2] = x,v,b #-v not v probably
        yield M, (2*b-v-x).reshape(-1,1)

In [3]:
a,b = next(bounce_att_gen(batch_size=2))

NameError: name 'bounce_att_gen' is not defined

In [4]:
i_b = Input(shape=(3,))

h_b = Dense(1, use_bias=False)(i_b)

out_b = h_b
# out = Dense(1, use_bias=False)(mul)
bounce = Model(inputs=i_b, outputs=out_b)
bounce.compile(loss=mean_squared_error,
              optimizer='adam',
              metrics=['accuracy'])
bounce.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 3)                 0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 3         
Total params: 3
Trainable params: 3
Non-trainable params: 0
_________________________________________________________________


In [11]:
bounce.fit_generator(bounce_gen(), steps_per_epoch=12800, epochs=5)

Epoch 1/5
12800/12800 [==============================] - 11s 867us/step - loss: 3.9842e-07 - acc: 0.0000e+00
Epoch 2/5
12800/12800 [==============================] - 11s 831us/step - loss: 1.7155e-07 - acc: 6.1035e-07
Epoch 3/5
12800/12800 [==============================] - 11s 834us/step - loss: 1.3642e-07 - acc: 6.1035e-07
Epoch 4/5
12800/12800 [==============================] - 11s 832us/step - loss: 1.2673e-07 - acc: 0.0000e+00
Epoch 5/5
12800/12800 [==============================] - 11s 825us/step - loss: 1.2023e-07 - acc: 0.0000e+00


In [17]:
save_model(bounce, 'bounce')

Saved model to disk


In [18]:
a, b = next(bounce_gen(batch_size=1))

In [19]:
a

array([[6.38947263, 0.0081253 , 6.39198096]])

In [20]:
bounce.predict(a)

array([[6.3863626]], dtype=float32)

In [21]:
b

array([[6.38636399]])

In [22]:
def move_gen(batch_size=128):
    while True:
        M = np.zeros((batch_size, 3))
        b = np.random.uniform(-10,10,size=batch_size)
        v = np.sign(b) * np.random.uniform(0, 0.1, size=batch_size)
        x = np.random.uniform(b-v, b)
        M[:,0],M[:,1],M[:,2] = x,v,b
        yield M, (x+v).reshape(-1,1)

In [23]:
i_b = Input(shape=(3,))

h_b = Dense(1, use_bias=False)(i_b)

out_b = h_b
# out = Dense(1, use_bias=False)(mul)
move = Model(inputs=i_b, outputs=out_b)
move.compile(loss=mean_squared_error,
              optimizer='adam',
              metrics=['accuracy'])
move.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 3)                 0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 3         
Total params: 3
Trainable params: 3
Non-trainable params: 0
_________________________________________________________________


In [26]:
move.fit_generator(move_gen(), steps_per_epoch=12800, epochs=20)

Epoch 1/20
12800/12800 [==============================] - 11s 887us/step - loss: 5.0896e-04 - acc: 6.1035e-07
Epoch 2/20
12800/12800 [==============================] - 11s 838us/step - loss: 4.2569e-04 - acc: 6.1035e-07
Epoch 3/20
12800/12800 [==============================] - 11s 839us/step - loss: 3.4703e-04 - acc: 6.1035e-07
Epoch 4/20
12800/12800 [==============================] - 11s 849us/step - loss: 2.8065e-04 - acc: 0.0000e+00
Epoch 5/20
12800/12800 [==============================] - 11s 836us/step - loss: 2.2179e-04 - acc: 1.8311e-06
Epoch 6/20
12800/12800 [==============================] - 11s 850us/step - loss: 1.7216e-04 - acc: 0.0000e+00
Epoch 7/20
12800/12800 [==============================] - 11s 834us/step - loss: 1.3083e-04 - acc: 0.0000e+00
Epoch 8/20
12800/12800 [==============================] - 11s 837us/step - loss: 9.6445e-05 - acc: 0.0000e+00
Epoch 9/20
12800/12800 [==============================] - 11s 840us/step - loss: 6.9087e-05 - acc: 0.0000e+00
Epoch 10/2

In [27]:
save_model(move, 'move')

Saved model to disk


In [32]:
def att_gen(batch_size=128):
    while True:
        M = np.zeros((batch_size, 3))
        b = np.random.uniform(-10,10,size=batch_size)
        v = np.sign(b) * np.random.uniform(0, 0.1, size=batch_size)
        x = np.random.uniform(b-2*v, b)
        M[:,0],M[:,1],M[:,2] = x,v,b
        yield M, ((np.abs(x+v)-np.abs(b))>0).reshape(-1,1)

In [142]:
a, b = next(att_gen(batch_size=2))

In [36]:
a,b

(array([[-9.10307176, -0.05965562, -9.18440158],
        [-0.10199596, -0.09809767, -0.18021052]]), array([[False],
        [ True]]))

In [37]:
a[1,0] + a[1,1]

-0.20009362358140548

In [75]:
change2 = open_and_load('change')

In [81]:
change2.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 1)                 0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 2         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 2         
Total params: 4
Trainable params: 4
Non-trainable params: 0
_________________________________________________________________


In [120]:
i_att = Input(shape=(3,))

a1 = Dense(2)(i_att)
a1_1 = Lambda(lambda s: [s[:,0]], output_shape=(1,))(a1)
a1_1 = Reshape((1,))(a1_1)
a2 = change2(a1_1)
mul1 = Multiply()([a1,a2])
s1 = Lambda(lambda s: s[:,0]+s[:,1], output_shape=(1,))(mul1)
s1 = Reshape((1,))(s1)

b1 = Dense(2)(i_att)
b1_1 = Lambda(lambda s: [s[:,0]], output_shape=(1,))(b1)
b1_1 = Reshape((1,))(b1_1)
b2 = change2(b1_1)
mul2 = Multiply()([b1,b2])
s2 = Lambda(lambda s: s[:,0]+s[:,1], output_shape=(1,))(mul2)
s2 = Reshape((1,))(s2)

a = Concatenate(axis=1)([s1,s2])
a = Dense(1, activation='relu')(a)
a = Dense(1, activation='hard_sigmoid')(a)

# out = Dense(1, use_bias=False)(mul)
att = Model(inputs=i_att, outputs=a)
att.compile(loss=mean_squared_error,
              optimizer='adam',
              metrics=['accuracy'])
att.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_49 (InputLayer)           (None, 3)            0                                            
__________________________________________________________________________________________________
dense_92 (Dense)                (None, 2)            8           input_49[0][0]                   
__________________________________________________________________________________________________
dense_93 (Dense)                (None, 2)            8           input_49[0][0]                   
__________________________________________________________________________________________________
lambda_63 (Lambda)              (None, 1)            0           dense_92[0][0]                   
__________________________________________________________________________________________________
lambda_65 

In [127]:
att.fit_generator(att_gen(), steps_per_epoch=12800, epochs=5)


Epoch 1/5
12800/12800 [==============================] - 31s 2ms/step - loss: 0.0397 - acc: 0.9491
Epoch 2/5
12800/12800 [==============================] - 30s 2ms/step - loss: 0.0393 - acc: 0.9496
Epoch 3/5
12800/12800 [==============================] - 30s 2ms/step - loss: 0.0405 - acc: 0.9482
Epoch 4/5
12800/12800 [==============================] - 30s 2ms/step - loss: 0.0393 - acc: 0.9496
Epoch 5/5
12800/12800 [==============================] - 30s 2ms/step - loss: 0.0397 - acc: 0.9491


In [137]:
a,b = next(att_gen(batch_size=1))

In [138]:
a,b

(array([[9.80013031e+00, 8.98551365e-03, 9.81086592e+00]]), array([[False]]))

In [139]:
att.predict(a)

array([[1.]], dtype=float32)

In [140]:
a[0,0]+a[0,1]

9.809115822537809

### 3 together

In [141]:
save_model(att, 'att3')

Saved model to disk


In [ ]:
i_att = Input(shape=(3,))

a1 = Dense(2)(i_att)
a1_1 = Lambda(lambda s: [s[:,0]], output_shape=(1,))(a1)
a1_1 = Reshape((1,))(a1_1)
a2 = change2(a1_1)
mul1 = Multiply()([a1,a2])
s1 = Lambda(lambda s: s[:,0]+s[:,1], output_shape=(1,))(mul1)
s1 = Reshape((1,))(s1)

b1 = Dense(2)(i_att)
b1_1 = Lambda(lambda s: [s[:,0]], output_shape=(1,))(b1)
b1_1 = Reshape((1,))(b1_1)
b2 = change2(b1_1)
mul2 = Multiply()([b1,b2])
s2 = Lambda(lambda s: s[:,0]+s[:,1], output_shape=(1,))(mul2)
s2 = Reshape((1,))(s2)

a = Concatenate(axis=1)([s1,s2])
a = Dense(1, activation='relu')(a)
a = Dense(1, activation='hard_sigmoid')(a)

# out = Dense(1, use_bias=False)(mul)
att = Model(inputs=i_att, outputs=a)
att.compile(loss=mean_squared_error,
              optimizer='adam',
              metrics=['accuracy'])
att.summary()